In [2]:
import json
import io
from datetime import datetime
import sys
import os
from functools import reduce
import openai

openai.api_key = os.environ['CHAT_GPT_TOKEN']
datetime.now().strftime("%Y-%m-%d %H:%M:%S")

'2023-01-10 22:31:34'

In [3]:
def get_summary(text: str, max_tokens, temperature=0.9, engine="davinci") -> str:
    max_total_tokens = 2049
    text = f"Please summarize the following text in {max_tokens} tokens: \n\n[{text}]"
    chunks = [text[i:i+max_total_tokens] for i in range(0, len(text), max_total_tokens)]
    max_summary_tokens = int(max_tokens / len(chunks))
    print(f"max_summary_tokens: {max_summary_tokens}")
    summaries = []
    for chunk in chunks:
        print(f"chunk: {chunk}")
        response = openai.Completion.create(
            engine=engine,
            prompt=chunk,
            max_tokens=max_summary_tokens,
            temperature=temperature,
            top_p=1
        )
        print("response: ", response)
        summaries.append(response.choices[0].text)
    summary = " ".join(summaries)
    return summary

print(datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

2023-01-10 22:31:34


In [4]:
text = "This is a test of the emergency broadcast system. This is only a test. If this had been an actual emergency, you would have been instructed to do something. This concludes this test of the emergency broadcast system."
get_summary(text, 100, temperature=0.9, engine="text-davinci-003")

max_summary_tokens: 100
chunk: Please summarize the following text in 100 tokens: 

[This is a test of the emergency broadcast system. This is only a test. If this had been an actual emergency, you would have been instructed to do something. This concludes this test of the emergency broadcast system.]
response:  {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "\n\nThis was a test of the emergency broadcast system. No actual emergency occurred. It was just a test. The test has now concluded."
    }
  ],
  "created": 1673418695,
  "id": "cmpl-6XOw3zLy2rwIowbWFvDLIdqW7XBuv",
  "model": "text-davinci-003",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 29,
    "prompt_tokens": 56,
    "total_tokens": 85
  }
}


'\n\nThis was a test of the emergency broadcast system. No actual emergency occurred. It was just a test. The test has now concluded.'

In [5]:
def split_text(text: str, max_chars: int=2049) -> list:

    words = text.split()
    groups = []
    current_group = ""

    for word in words:
        if len(current_group) < max_chars:
            current_group += " " + word
        else:
            groups.append(current_group[1:])
            current_group = " " + word

    return groups

text = "Lorem ipsum dolor sit amet, consectetur adipiscing elit. Sed tincidunt ornare ipsum, a pretium orci malesuada eu. Suspendisse non sollicitudin velit, at ullamcorper urna. Donec et enim dapibus, elementum lectus et, tincidunt purus. Vivamus ultricies, turpis ut elementum interdum, purus dolor consectetur urna, at euismod nulla libero non purus. Nullam ac dui id ex imperdiet aliquam. Maecenas in luctus massa. Nulla facilisi. Fusce venenatis interdum tortor non laoreet. Donec lacinia ex quis vehicula elementum. Integer vel pretium velit. Morbi scelerisque tellus et dolor porttitor scelerisque. Suspendisse euismod, sem sit amet vehicula interdum, erat diam rutrum elit, eu tincidunt mauris justo sed tellus."

groups = split_text(text, 10)
print(groups)


['Lorem ipsum', 'dolor sit', 'amet, consectetur', 'adipiscing', 'elit. Sed', 'tincidunt', 'ornare ipsum,', 'a pretium', 'orci malesuada', 'eu. Suspendisse', 'non sollicitudin', 'velit, at', 'ullamcorper', 'urna. Donec', 'et enim dapibus,', 'elementum', 'lectus et,', 'tincidunt', 'purus. Vivamus', 'ultricies,', 'turpis ut', 'elementum', 'interdum,', 'purus dolor', 'consectetur', 'urna, at euismod', 'nulla libero', 'non purus.', 'Nullam ac', 'dui id ex', 'imperdiet', 'aliquam. Maecenas', 'in luctus', 'massa. Nulla', 'facilisi.', 'Fusce venenatis', 'interdum tortor', 'non laoreet.', 'Donec lacinia', 'ex quis vehicula', 'elementum.', 'Integer vel', 'pretium velit.', 'Morbi scelerisque', 'tellus et', 'dolor porttitor', 'scelerisque.', 'Suspendisse', 'euismod, sem', 'sit amet vehicula', 'interdum,', 'erat diam', 'rutrum elit,', 'eu tincidunt', 'mauris justo']


In [6]:
import re

def split_text(text: str) -> list:
    # Split the text into sentences
    sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', text)

    # Split each sentence into words and return a nested list
    return [sentence.split() for sentence in sentences]

text = "Lorem ipsum dolor sit amet, consectetur adipiscing elit. Sed tincidunt ornare ipsum, a pretium orci malesuada eu. Suspendisse non sollicitudin velit, at ullamcorper urna. Donec et enim dapibus, elementum lectus et, tincidunt purus. Vivamus ultricies, turpis ut elementum interdum, purus dolor consectetur urna, at euismod nulla libero non purus. Nullam ac dui id ex imperdiet aliquam. Maecenas in luctus massa. Nulla facilisi. Fusce venenatis interdum tortor non laoreet. Donec lacinia ex quis vehicula elementum. Integer vel pretium velit. Morbi scelerisque tellus et dolor porttitor scelerisque. Suspendisse euismod, sem sit amet vehicula interdum, erat diam rutrum elit, eu tincidunt mauris justo sed tellus."

sentences_and_words = split_text(text)
print(sentences_and_words)


[['Lorem', 'ipsum', 'dolor', 'sit', 'amet,', 'consectetur', 'adipiscing', 'elit.'], ['Sed', 'tincidunt', 'ornare', 'ipsum,', 'a', 'pretium', 'orci', 'malesuada', 'eu.'], ['Suspendisse', 'non', 'sollicitudin', 'velit,', 'at', 'ullamcorper', 'urna.'], ['Donec', 'et', 'enim', 'dapibus,', 'elementum', 'lectus', 'et,', 'tincidunt', 'purus.'], ['Vivamus', 'ultricies,', 'turpis', 'ut', 'elementum', 'interdum,', 'purus', 'dolor', 'consectetur', 'urna,', 'at', 'euismod', 'nulla', 'libero', 'non', 'purus.'], ['Nullam', 'ac', 'dui', 'id', 'ex', 'imperdiet', 'aliquam.'], ['Maecenas', 'in', 'luctus', 'massa.'], ['Nulla', 'facilisi.'], ['Fusce', 'venenatis', 'interdum', 'tortor', 'non', 'laoreet.'], ['Donec', 'lacinia', 'ex', 'quis', 'vehicula', 'elementum.'], ['Integer', 'vel', 'pretium', 'velit.'], ['Morbi', 'scelerisque', 'tellus', 'et', 'dolor', 'porttitor', 'scelerisque.'], ['Suspendisse', 'euismod,', 'sem', 'sit', 'amet', 'vehicula', 'interdum,', 'erat', 'diam', 'rutrum', 'elit,', 'eu', 'tinci

In [7]:
chunk = "What is a final variable declaration in Java"
engine = 'text-davinci-003'
max_summary_tokens = 4000
temperature = 0.9
response = openai.Completion.create(
    engine=engine,
    prompt=chunk,
    max_tokens=max_summary_tokens,
    temperature=temperature,
    top_p=1
)
print("response: ", response)

response:  {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "?\n\nA final variable declaration is a declaration in Java that the value of a variable cannot be modified after the initial assignment. Final variables must be assigned a value at the time of declaration and cannot be reassigned later. This is useful for constants or other values that should always be static."
    }
  ],
  "created": 1673418704,
  "id": "cmpl-6XOwC9jgjR5ctrLfrihH3VbT9pygR",
  "model": "text-davinci-003",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 59,
    "prompt_tokens": 8,
    "total_tokens": 67
  }
}
